In [ ]:
import osmnx as ox

place_name = "Coimbatore, India"
tags = {"amenity": "police"}  # Fetch police station locations

# Correct function
gdf_police = ox.features_from_place(place_name, tags=tags)
display(gdf_police)
gdf_police.to_excel("coimbatore_police.xlsx", index=False)


In [35]:
import osmnx as ox

place_name = "Coimbatore, India"
tags = {"amenity": "police"}  # Fetch police station locations

# Fetch police station data
gdf_police = ox.features_from_place(place_name, tags=tags)

# Extract OSM node IDs as a list
node_ids = gdf_police.index.tolist()

# Print the list of node IDs
print(node_ids)


[('node', 247177690), ('node', 308446629), ('node', 308449858), ('node', 803271976), ('node', 803272013), ('node', 1051328308), ('node', 1051328344), ('node', 1051328360), ('node', 1051328368), ('node', 1153096222), ('node', 1423854834), ('node', 1470432055), ('node', 1471854713), ('node', 1474350853), ('node', 1474518443), ('node', 1474803271), ('node', 1680232798), ('node', 3571694093), ('node', 4450449989), ('node', 5338402894), ('node', 5970940185), ('node', 11411647470), ('node', 12122699748), ('node', 12473835683), ('node', 12473896167), ('node', 12473897891), ('node', 12473901192), ('node', 12473949936), ('node', 12473970775), ('node', 12473983454), ('way', 130035512), ('way', 133585587), ('way', 133739505), ('way', 295103984), ('way', 580408779), ('way', 685342422), ('way', 923519789), ('way', 1149353353), ('way', 1149353354), ('way', 1149353355), ('way', 1149353356), ('way', 1149353357), ('way', 1237329086), ('way', 1302295330), ('way', 1348530473)]


In [ ]:
G = ox.graph_from_place("Coimbatore, India", network_type="drive")
boundary_gdf = ox.geocode_to_gdf("Coimbatore, India")
import leafmap.kepler as leafmap

m = leafmap.Map(center=[11.0168, 76.9558], zoom=12)
m.add_gdf(boundary_gdf, layer_name="Coimbatore Boundary")
m.add_graph(G, layer_name="Road Network")
# or just show the boundary
m.add_gdf(
    boundary_gdf, 
    layer_name="Boundary",
    style={"color": "#FF0000", "fillOpacity": 0.1}
)



# Comparision between ALGORITHMS

In [13]:
# --------------------------------------------------------------------------------
# CELL 1: Imports and Basic Setup
# --------------------------------------------------------------------------------

import os
import time
import random
import numpy as np
import pandas as pd
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
from tqdm.notebook import tqdm

# Make sure these are installed:
# !pip install osmnx networkx shapely geopandas tqdm pandas

ox.settings.use_cache = True
ox.settings.log_console = False


In [14]:

# --------------------------------------------------------------------------------
# CELL 2: Load Road Network (Coimbatore) and Basic Helpers
# --------------------------------------------------------------------------------

def load_road_network(place="Coimbatore, India", cache_dir="road_cache"):
    """
    Loads (or downloads) the road network for the specified place into a NetworkX graph.
    Returns a directed graph for 'drive' usage.
    """
    os.makedirs(cache_dir, exist_ok=True)
    safe_place = place.replace(" ", "_").replace(",", "")
    cache_file = os.path.join(cache_dir, f"{safe_place}_network.graphml")
    path_file = r"C:\Users\DELL\Documents\Amrita\4th year\ArcGis\major_road_cache\Coimbatore_India_major.graphml"
    if os.path.exists(path_file):
        print(f"Loading cached road network from {cache_file}")
        G = ox.load_graphml(cache_file)
    else:
        print(f"Downloading road network for {place}...")
        G = ox.graph_from_place(place, network_type="drive", simplify=True)
        ox.save_graphml(G, cache_file)
    return G

def count_edge_usage(G, solution_routes):
    """
    For a list of routes (each route is a list of nodes),
    return a dictionary edge_usage such that edge_usage[(u, v)] = number_of_vehicles_that_used_this_edge.
    Also returns the set of all covered edges.
    """
    edge_usage = {}
    covered_edges = set()
    for route in solution_routes:
        for u, v in zip(route[:-1], route[1:]):
            # Because G is directed, store usage in the directed sense
            if G.has_edge(u, v):
                edge_usage[(u, v)] = edge_usage.get((u, v), 0) + 1
                covered_edges.add((u, v))
            else:
                # If the route references an edge not in G or reversed, do a small fallback:
                if G.has_edge(v, u):
                    edge_usage[(v, u)] = edge_usage.get((v, u), 0) + 1
                    covered_edges.add((v, u))
                else:
                    # Edge not found at all
                    pass
    return edge_usage, covered_edges

def coverage_fitness(
    G, 
    solution_routes, 
    coverage_factor=5000, 
    overlap_penalty_factor=10000,
    max_overlap=3
):
    """
    A standard coverage-based fitness:
     - coverage_reward = coverage_factor * sum_of_edge_length_for_all_covered_edges
     - If any edge is used by > max_overlap vehicles, we subtract an overlap penalty.
    """
    edge_usage, covered_edges = count_edge_usage(G, solution_routes)
    
    # Sum coverage reward
    coverage_reward = 0.0
    for (u,v) in covered_edges:
        length = G[u][v][0].get('length', 0.0) if G.has_edge(u,v) else 0.0
        coverage_reward += coverage_factor * length
    
    # Overlap penalty
    overlap_penalty = 0.0
    for (e, usage_count) in edge_usage.items():
        if usage_count > max_overlap:
            length = G[e[0]][e[1]][0].get('length', 0.0)
            overlap_penalty += overlap_penalty_factor * (usage_count - max_overlap) * length
    
    return coverage_reward - overlap_penalty

def route_to_random(G, max_dist=15000):
    """
    Creates a single random route by starting at a random node, 
    walking along random successors until we exceed max_dist or have no successors.
    """
    nodes_list = list(G.nodes())
    if not nodes_list:
        return []
    
    current = random.choice(nodes_list)
    route = [current]
    dist_so_far = 0.0
    
    while True:
        neighbors = list(G.successors(current))
        if not neighbors:
            break
        nxt = random.choice(neighbors)
        length = G[current][nxt][0].get('length', 0.0)
        if dist_so_far + length > max_dist:
            break
        route.append(nxt)
        dist_so_far += length
        current = nxt
    return route

# We'll unify a route-building for PSO+RRT or ACO+RRT by a small "random expansions" approach:
def route_rrt_style(G, max_dist=15000, max_steps=20):
    """
    A simplistic RRT-like approach: start from a random node, pick random nodes,
    connect by shortest paths, until distance limit or max_steps is reached.
    """
    import math
    
    nodes_list = list(G.nodes())
    if not nodes_list:
        return []
    current = random.choice(nodes_list)
    route = [current]
    dist_so_far = 0.0
    
    for _ in range(max_steps):
        target = random.choice(nodes_list)
        # Attempt shortest path from current -> target
        if current == target:
            continue
        try:
            path = nx.shortest_path(G, current, target, weight='length')
        except nx.NetworkXNoPath:
            continue
        # sum subpath length
        sub_len = 0.0
        for u,v in zip(path[:-1], path[1:]):
            sub_len += G[u][v][0].get('length', 0.0)
        if dist_so_far + sub_len > max_dist:
            break
        # append path (excluding repeated current)
        route.extend(path[1:])
        dist_so_far += sub_len
        current = target
    return route


In [15]:

# --------------------------------------------------------------------------------
# CELL 3: Genetic Algorithm (GA)
# --------------------------------------------------------------------------------

def run_ga(
    G,
    num_vehicles=5,
    population_size=20,
    num_generations=30,
    coverage_factor=5000,
    overlap_penalty_factor=10000,
    max_dist=15000
):
    """
    Simple GA that evolves a population of solutions. Each solution is a list of `num_vehicles` routes.
    We'll track iteration-wise best fitness, final best solution, final best fitness, etc.
    """
    # Create initial population
    def build_solution():
        return [route_to_random(G, max_dist=max_dist) for _ in range(num_vehicles)]
    
    def fitness(solution):
        return coverage_fitness(G, solution, coverage_factor, overlap_penalty_factor)
    
    population = [build_solution() for _ in range(population_size)]
    
    best_fitness_history = []
    best_solution = None
    best_val = -float('inf')
    
    start_time = time.time()
    
    for _ in tqdm(range(num_generations),desc="GA Generation"):
        # Evaluate fitness
        fits = [fitness(sol) for sol in population]
        
        # Track best
        for i, val in enumerate(fits):
            if val > best_val:
                best_val = val
                best_solution = population[i]
        
        best_fitness_history.append(best_val)
        
        # Selection (tournament)
        new_pop = []
        for _ in range(population_size):
            a, b = random.sample(range(population_size), 2)
            if fits[a] > fits[b]:
                new_pop.append(population[a])
            else:
                new_pop.append(population[b])
        
        # Crossover & mutation
        next_gen = []
        for i in range(0, population_size, 2):
            p1 = new_pop[i]
            p2 = new_pop[i+1] if i+1 < population_size else new_pop[0]
            
            # Single-point crossover on routes
            child1 = []
            child2 = []
            for v in range(num_vehicles):
                if random.random() < 0.5:
                    child1.append(p1[v])
                    child2.append(p2[v])
                else:
                    child1.append(p2[v])
                    child2.append(p1[v])
            
            # Mutation: randomly rebuild one route in child
            if random.random() < 0.3:
                idx = random.randint(0, num_vehicles-1)
                child1[idx] = route_to_random(G, max_dist=max_dist)
            if random.random() < 0.3:
                idx = random.randint(0, num_vehicles-1)
                child2[idx] = route_to_random(G, max_dist=max_dist)
            
            next_gen.append(child1)
            next_gen.append(child2)
        
        population = next_gen[:population_size]
    
    end_time = time.time()
    run_time = end_time - start_time
    
    # Count edges usage for final solution
    edge_usage, _ = count_edge_usage(G, best_solution)
    return {
        "algorithm": "GA",
        "best_solution": best_solution,
        "best_fitness": best_val,
        "fitness_history": best_fitness_history,
        "time_secs": run_time,
        "edge_usage": edge_usage
    }


In [16]:

# --------------------------------------------------------------------------------
# CELL 4: Particle Swarm Optimization (PSO)
# --------------------------------------------------------------------------------

def run_pso(
    G,
    num_vehicles=5,
    swarm_size=20,
    num_iterations=30,
    coverage_factor=5000,
    overlap_penalty_factor=10000,
    max_dist=15000
):
    """
    Discrete PSO approach: each 'particle' is a set of routes. We'll do a
    naive velocity update by partial adoption of personal/global best.
    """
    def build_solution():
        return [route_to_random(G, max_dist=max_dist) for _ in range(num_vehicles)]
    
    def fitness(solution):
        return coverage_fitness(G, solution, coverage_factor, overlap_penalty_factor)
    
    # Initialize
    swarm = [build_solution() for _ in range(swarm_size)]
    personal_best = list(swarm)
    personal_fit = [fitness(sol) for sol in swarm]
    
    global_best = None
    global_best_fit = -float('inf')
    for i in tqdm(range(swarm_size),desc="PSO"):
        if personal_fit[i] > global_best_fit:
            global_best_fit = personal_fit[i]
            global_best = personal_best[i]
    
    best_fitness_history = []
    
    start_time = time.time()
    
    for _ in range(num_iterations):
        for i in range(swarm_size):
            # velocity update by partial adoption
            candidate = []
            for v in range(num_vehicles):
                r = random.random()
                if r < 0.34:
                    # keep current route
                    candidate.append(swarm[i][v])
                elif r < 0.67:
                    # adopt personal best route
                    candidate.append(personal_best[i][v])
                else:
                    # adopt global best route
                    candidate.append(global_best[v])
            
            # mutation: random chance to rebuild one route
            if random.random() < 0.3:
                idx = random.randint(0, num_vehicles-1)
                candidate[idx] = route_to_random(G, max_dist=max_dist)
            
            new_fit = fitness(candidate)
            old_fit = fitness(swarm[i])
            if new_fit > old_fit:
                swarm[i] = candidate
                if new_fit > personal_fit[i]:
                    personal_fit[i] = new_fit
                    personal_best[i] = candidate
                    if new_fit > global_best_fit:
                        global_best_fit = new_fit
                        global_best = candidate
        
        best_fitness_history.append(global_best_fit)
    
    end_time = time.time()
    run_time = end_time - start_time
    
    edge_usage, _ = count_edge_usage(G, global_best)
    return {
        "algorithm": "PSO",
        "best_solution": global_best,
        "best_fitness": global_best_fit,
        "fitness_history": best_fitness_history,
        "time_secs": run_time,
        "edge_usage": edge_usage
    }


In [17]:

# --------------------------------------------------------------------------------
# CELL 5: PSO + RRT* (We will just adapt the route-building function to RRT style)
# --------------------------------------------------------------------------------

def run_pso_rrt(
    G,
    num_vehicles=5,
    swarm_size=20,
    num_iterations=30,
    coverage_factor=5000,
    overlap_penalty_factor=10000,
    max_dist=15000
):
    def build_solution():
        return [route_rrt_style(G, max_dist=max_dist, max_steps=20) for _ in range(num_vehicles)]
    
    def fitness(solution):
        return coverage_fitness(G, solution, coverage_factor, overlap_penalty_factor)
    
    # Initialize
    swarm = [build_solution() for _ in range(swarm_size)]
    personal_best = list(swarm)
    personal_fit = [fitness(sol) for sol in swarm]
    
    global_best = None
    global_best_fit = -float('inf')
    for i in range(swarm_size):
        if personal_fit[i] > global_best_fit:
            global_best_fit = personal_fit[i]
            global_best = personal_best[i]
    
    best_fitness_history = []
    start_time = time.time()
    
    for _ in tqdm(range(num_iterations),desc="PSO+RRT"):
        for i in range(swarm_size):
            # velocity update
            candidate = []
            for v in range(num_vehicles):
                r = random.random()
                if r < 0.34:
                    candidate.append(swarm[i][v])
                elif r < 0.67:
                    candidate.append(personal_best[i][v])
                else:
                    candidate.append(global_best[v])
            
            # random mutation: rebuild one route with RRT
            if random.random() < 0.3:
                idx = random.randint(0, num_vehicles-1)
                candidate[idx] = route_rrt_style(G, max_dist=max_dist, max_steps=20)
            
            new_fit = fitness(candidate)
            old_fit = fitness(swarm[i])
            if new_fit > old_fit:
                swarm[i] = candidate
                if new_fit > personal_fit[i]:
                    personal_fit[i] = new_fit
                    personal_best[i] = candidate
                    if new_fit > global_best_fit:
                        global_best_fit = new_fit
                        global_best = candidate
        
        best_fitness_history.append(global_best_fit)
    
    end_time = time.time()
    run_time = end_time - start_time
    
    edge_usage, _ = count_edge_usage(G, global_best)
    return {
        "algorithm": "PSO+RRT",
        "best_solution": global_best,
        "best_fitness": global_best_fit,
        "fitness_history": best_fitness_history,
        "time_secs": run_time,
        "edge_usage": edge_usage
    }


In [18]:

# --------------------------------------------------------------------------------
# CELL 6: ACO + RRT* (Ant Colony approach, RRT-based routes)
# --------------------------------------------------------------------------------

def run_aco_rrt(
    G,
    num_vehicles=5,
    num_iterations=30,
    num_ants=10,           # how many ants per iteration
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.1,
    coverage_factor=5000,
    overlap_penalty_factor=10000,
    max_dist=15000
):
    """
    Very simple ACO that at each iteration builds 'num_ants' solutions with route_rrt_style,
    picks the best, deposits pheromone, etc. 
    (This is a toy example just to demonstrate a style.)
    """
    # We'll store pheromone on a per-edge basis
    # Initialize all edges with tau=1
    edges_list = list(G.edges())
    pheromone = { (u,v): 1.0 for (u,v) in edges_list }
    
    def fitness(solution):
        return coverage_fitness(G, solution, coverage_factor, overlap_penalty_factor)
    
    best_solution = None
    best_val = -float('inf')
    best_fitness_history = []
    
    start_time = time.time()
    
    for _ in tqdm(range(num_iterations),desc='aco_rrt'):
        iteration_bests = []
        solutions = []
        fits = []
        # Build solutions
        for _ant in range(num_ants):
            # For each ant, build 'num_vehicles' routes via RRT
            sol = [route_rrt_style(G, max_dist=max_dist, max_steps=20) for _ in range(num_vehicles)]
            val = fitness(sol)
            solutions.append(sol)
            fits.append(val)
        
        # Identify iteration best
        idx_best = np.argmax(fits)
        iteration_best_sol = solutions[idx_best]
        iteration_best_val = fits[idx_best]
        
        if iteration_best_val > best_val:
            best_val = iteration_best_val
            best_solution = iteration_best_sol
        
        best_fitness_history.append(best_val)
        
        # Evaporate pheromone
        for e in pheromone:
            pheromone[e] *= (1 - evaporation_rate)
            if pheromone[e] < 0:
                pheromone[e] = 0
        
        # Deposit from iteration best
        deposit_amount = max(iteration_best_val, 1)
        # We'll do a minimal deposit on all edges used
        eu, _ = count_edge_usage(G, iteration_best_sol)
        for e, usage_count in eu.items():
            # deposit is scaled by usage_count
            pheromone[e] = pheromone.get(e, 0) + usage_count * (deposit_amount / 1000.0)
    
    end_time = time.time()
    run_time = end_time - start_time
    
    edge_usage, _ = count_edge_usage(G, best_solution)
    return {
        "algorithm": "ACO+RRT",
        "best_solution": best_solution,
        "best_fitness": best_val,
        "fitness_history": best_fitness_history,
        "time_secs": run_time,
        "edge_usage": edge_usage
    }


In [19]:

# --------------------------------------------------------------------------------
# CELL 7: ASRank (Rank-Based ACO) – simplified
# --------------------------------------------------------------------------------

def run_asrank(
    G,
    num_vehicles=5,
    num_iterations=30,
    top_w=5,               # top w solutions deposit
    coverage_factor=5000,
    overlap_penalty_factor=10000,
    population_per_iter=10,
    evaporation_rate=0.1,
    alpha=1.0,
    beta=2.0,
    max_dist=15000
):
    """
    Each iteration builds 'population_per_iter' solutions (random or RRT?), ranks them,
    top w deposit pheromone with rank-based weighting.
    """
    edges_list = list(G.edges())
    pheromone = { (u,v): 1.0 for (u,v) in edges_list }
    
    def fitness(solution):
        return coverage_fitness(G, solution, coverage_factor, overlap_penalty_factor)
    
    best_solution = None
    best_val = -float('inf')
    best_fitness_history = []
    
    start_time = time.time()
    
    for _ in tqdm(range(num_iterations),desc='asrank'):
        iteration_solutions = []
        iteration_fits = []
        
        # build population_per_iter solutions
        for _sol_i in range(population_per_iter):
            sol = [route_to_random(G, max_dist=max_dist) for _ in range(num_vehicles)]
            val = fitness(sol)
            iteration_solutions.append(sol)
            iteration_fits.append(val)
        
        # rank them
        idx_sorted = np.argsort(iteration_fits)[::-1]
        if iteration_fits[idx_sorted[0]] > best_val:
            best_val = iteration_fits[idx_sorted[0]]
            best_solution = iteration_solutions[idx_sorted[0]]
        
        best_fitness_history.append(best_val)
        
        # Evaporate
        for e in pheromone:
            pheromone[e] *= (1 - evaporation_rate)
            if pheromone[e] < 0:
                pheromone[e] = 0
        
        # deposit from top w
        for rank_i in range(min(top_w, population_per_iter)):
            idx = idx_sorted[rank_i]
            rank_weight = (top_w - rank_i)  # or some scale
            sol = iteration_solutions[idx]
            val = iteration_fits[idx]
            
            deposit = max(val, 1) * rank_weight
            eu, _ = count_edge_usage(G, sol)
            for e, usage_count in eu.items():
                pheromone[e] = pheromone.get(e, 0) + (usage_count * deposit / 1000.0)
    
    end_time = time.time()
    run_time = end_time - start_time
    
    edge_usage, _ = count_edge_usage(G, best_solution)
    return {
        "algorithm": "ASRank",
        "best_solution": best_solution,
        "best_fitness": best_val,
        "fitness_history": best_fitness_history,
        "time_secs": run_time,
        "edge_usage": edge_usage
    }


In [20]:

# --------------------------------------------------------------------------------
# CELL 8: MMAS (MAX-MIN Ant System) – simplified
# --------------------------------------------------------------------------------

def run_mmas(
    G,
    num_vehicles=5,
    num_iterations=30,
    ants_per_iter=10,
    coverage_factor=5000,
    overlap_penalty_factor=10000,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.1,
    tau_min=0.1,
    tau_max=5.0,
    max_dist=15000
):
    edges_list = list(G.edges())
    # init pheromone
    pheromone = { e: 1.0 for e in edges_list }
    
    def fitness(solution):
        return coverage_fitness(G, solution, coverage_factor, overlap_penalty_factor)
    
    best_sol = None
    best_val = -float('inf')
    best_fitness_history = []
    
    start_time = time.time()
    
    for _ in tqdm(range(num_iterations),desc='mmas'):
        # build solutions
        iteration_solutions = []
        iteration_fits = []
        for _ant in range(ants_per_iter):
            sol = [route_to_random(G, max_dist=max_dist) for _ in range(num_vehicles)]
            val = fitness(sol)
            iteration_solutions.append(sol)
            iteration_fits.append(val)
        # find iteration best
        ibest_idx = np.argmax(iteration_fits)
        ibest_val = iteration_fits[ibest_idx]
        ibest_sol = iteration_solutions[ibest_idx]
        
        if ibest_val > best_val:
            best_val = ibest_val
            best_sol = ibest_sol
        
        best_fitness_history.append(best_val)
        
        # Evap
        for e in pheromone:
            pheromone[e] *= (1 - evaporation_rate)
            if pheromone[e] < 0:
                pheromone[e] = 0
        
        # deposit from global best
        deposit_amt = max(best_val, 1)
        eusage, _ = count_edge_usage(G, best_sol)
        for e, usage_count in eusage.items():
            pheromone[e] = pheromone.get(e, 0) + (usage_count * deposit_amt / 1000.0)
        
        # clamp to [tau_min, tau_max]
        for e in pheromone:
            if pheromone[e] < tau_min:
                pheromone[e] = tau_min
            elif pheromone[e] > tau_max:
                pheromone[e] = tau_max
    
    end_time = time.time()
    run_time = end_time - start_time
    
    edge_usage, _ = count_edge_usage(G, best_sol)
    return {
        "algorithm": "MMAS",
        "best_solution": best_sol,
        "best_fitness": best_val,
        "fitness_history": best_fitness_history,
        "time_secs": run_time,
        "edge_usage": edge_usage
    }


In [21]:

# --------------------------------------------------------------------------------
# CELL 9: Main Experiment – multiple seeds & gather results
# --------------------------------------------------------------------------------

# We will run each algorithm with 2 random seeds. That gives us 14 runs 
# (7 algorithms * 2 seeds). Then we can show the top 10 by best fitness.

def main_experiment():
    # Step 1: Load graph
    G = load_road_network("Coimbatore, India")
    
    # For fairness, keep all coverage params the same
    coverage_factor = 5000
    overlap_penalty_factor = 10000
    num_vehicles = 5
    # We'll use a max_dist of 15000, iteration count 30, etc.
    
    results = []
    seeds = [42, 999]   # do 2 seeds for demonstration
    
    for seed in tqdm(seeds,desc="Seed"):
        random.seed(seed)
        np.random.seed(seed)
        
        # GA
        ga_res = run_ga(
            G=G,
            num_vehicles=num_vehicles,
            population_size=20,
            num_generations=30,
            coverage_factor=coverage_factor,
            overlap_penalty_factor=overlap_penalty_factor,
            max_dist=15000
        )
        ga_res["seed"] = seed
        results.append(ga_res)
        
        # PSO
        pso_res = run_pso(
            G=G,
            num_vehicles=num_vehicles,
            swarm_size=20,
            num_iterations=30,
            coverage_factor=coverage_factor,
            overlap_penalty_factor=overlap_penalty_factor,
            max_dist=15000
        )
        pso_res["seed"] = seed
        results.append(pso_res)
        
        # PSO+RRT
        pso_rrt_res = run_pso_rrt(
            G=G,
            num_vehicles=num_vehicles,
            swarm_size=20,
            num_iterations=30,
            coverage_factor=coverage_factor,
            overlap_penalty_factor=overlap_penalty_factor,
            max_dist=15000
        )
        pso_rrt_res["seed"] = seed
        results.append(pso_rrt_res)
        
        # ACO+RRT
        aco_rrt_res = run_aco_rrt(
            G=G,
            num_vehicles=num_vehicles,
            num_iterations=30,
            num_ants=10,
            alpha=1.0,
            beta=2.0,
            evaporation_rate=0.1,
            coverage_factor=coverage_factor,
            overlap_penalty_factor=overlap_penalty_factor,
            max_dist=15000
        )
        aco_rrt_res["seed"] = seed
        results.append(aco_rrt_res)
        
        # ASRank
        asrank_res = run_asrank(
            G=G,
            num_vehicles=num_vehicles,
            num_iterations=30,
            top_w=5,
            coverage_factor=coverage_factor,
            overlap_penalty_factor=overlap_penalty_factor,
            population_per_iter=10,
            evaporation_rate=0.1,
            alpha=1.0,
            beta=2.0,
            max_dist=15000
        )
        asrank_res["seed"] = seed
        results.append(asrank_res)
        
        # MMAS
        mmas_res = run_mmas(
            G=G,
            num_vehicles=num_vehicles,
            num_iterations=30,
            ants_per_iter=10,
            coverage_factor=coverage_factor,
            overlap_penalty_factor=overlap_penalty_factor,
            alpha=1.0,
            beta=2.0,
            evaporation_rate=0.1,
            tau_min=0.1,
            tau_max=5.0,
            max_dist=15000
        )
        mmas_res["seed"] = seed
        results.append(mmas_res)
    
    # Convert to DataFrame
    data_rows = []
    for r in results:
        data_rows.append({
            "Algorithm": r["algorithm"],
            "Seed": r["seed"],
            "BestFitness": r["best_fitness"],
            "TimeSecs": r["time_secs"],
            "FitnessHistory": r["fitness_history"],  # Could store or skip
            "NumEdgesUsed": len(r["edge_usage"]),
            # If you want, you can also track how many edges had >1 usage, etc.
            "OverlapCount": sum(1 for e,u in r["edge_usage"].items() if u>1)
        })
    df = pd.DataFrame(data_rows)
    
    # Sort by best fitness descending
    df_sorted = df.sort_values("BestFitness", ascending=False)
    # We'll show the top 10
    top10 = df_sorted.head(10)
    
    print("\n=== ALL RESULTS (sorted by BestFitness) ===")
    print(df_sorted)
    print("\n=== TOP 10 ===")
    print(top10)
    
    return df_sorted


In [22]:

# --------------------------------------------------------------------------------
# CELL 10: Actually run and show top 10
# --------------------------------------------------------------------------------

if __name__ == "__main__":
    final_df = main_experiment()
    # The final_df is sorted by best fitness and you have the top 10 printed out above.


Loading cached road network from road_cache\Coimbatore_India_network.graphml


Seed:   0%|          | 0/2 [00:00<?, ?it/s]

GA Generation:   0%|          | 0/30 [00:00<?, ?it/s]

PSO:   0%|          | 0/20 [00:00<?, ?it/s]

PSO+RRT:   0%|          | 0/30 [00:00<?, ?it/s]

...

PSO+RRT:   0%|          | 0/30 [00:00<?, ?it/s]

aco_rrt:   0%|          | 0/30 [00:00<?, ?it/s]

asrank:   0%|          | 0/30 [00:00<?, ?it/s]

mmas:   0%|          | 0/30 [00:00<?, ?it/s]


=== ALL RESULTS (sorted by BestFitness) ===
   Algorithm  Seed   BestFitness    TimeSecs  \
2    PSO+RRT    42  3.629562e+08   97.436647   
8    PSO+RRT   999  3.482209e+08   95.319434   
1        PSO    42  3.166337e+08    1.364410   
7        PSO   999  2.876853e+08    1.108380   
9    ACO+RRT   999  2.713497e+08  745.446122   
6         GA   999  2.700360e+08    0.989013   
0         GA    42  2.689422e+08    0.994706   
11      MMAS   999  2.372645e+08    9.594722   
3    ACO+RRT    42  2.280921e+08  801.476915   
4     ASRank    42  2.087888e+08    7.429187   
10    ASRank   999  2.080822e+08    7.660267   
5       MMAS    42  2.021233e+08    9.340113   

                                       FitnessHistory  NumEdgesUsed  \
2   [193916188.79291332, 231610432.7278762, 231610...           913   
8   [314775506.9202112, 314775506.9202112, 3225158...           709   
1   [210429144.77257937, 229724834.94366944, 22972...           186   
7   [209830926.8595423, 242045809.46627647, 25

In [25]:
final_df


,Algorithm,Seed,BestFitness,TimeSecs,FitnessHistory,NumEdgesUsed,OverlapCount
2,PSO+RRT,42,3.629562e+08,97.436647,"[193916188.79291332, 231610432.7278762, 231610...",913,0
8,PSO+RRT,999,3.482209e+08,95.319434,"[314775506.9202112, 314775506.9202112, 3225158...",709,0
1,PSO,42,3.166337e+08,1.364410,"[210429144.77257937, 229724834.94366944, 22972...",186,51
7,PSO,999,2.876853e+08,1.108380,"[209830926.8595423, 242045809.46627647, 253229...",215,55
9,ACO+RRT,999,2.713497e+08,745.446122,"[271349720.98737514, 271349720.98737514, 27134...",674,0
6,GA,999,2.700360e+08,0.989013,"[174532682.03882086, 222936153.00590655, 22293...",496,156
0,GA,42,2.689422e+08,0.994706,"[213787342.5052342, 213787342.5052342, 2137873...",387,136
11,MMAS,999,2.372645e+08,9.594722,"[194644876.75564927, 194644876.75564927, 19464...",465,200
3,ACO+RRT,42,2.280921e+08,801.476915,"[183490503.6284807, 183490503.6284807, 1834905...",449,0
4,ASRank,42,2.087888e+08,7.429187,"[195473582.0495971, 195473582.0495971, 1954735...",540,194
